# GO COCO - Encuesta Nacional de Hospitales Survey - Data Cleaning Script

2/16/2020

Hult International Business School

# 0. Overview

Angostura is hosted on Google's BigQuery.

This notebook will help you to instal the required dependencies and connect to Angostura's BigQuery database.

# 1. Install Dependencies

The only required dependency to communicate to BigQuery is the Google Cloud API.

In [2]:
import sys
!{sys.executable} -m pip install --upgrade pandas
!{sys.executable} -m pip install --upgrade google-cloud-bigquery[pandas]

Requirement already up-to-date: pandas in /Users/johnvillanueva/opt/anaconda3/envs/hackathon/lib/python3.7/site-packages (1.0.1)
Requirement already up-to-date: google-cloud-bigquery[pandas] in /Users/johnvillanueva/opt/anaconda3/envs/hackathon/lib/python3.7/site-packages (1.24.0)


# 2. Authentication Setup

Before being able to query the Angostura database you need to authenticate.
You will have read-only access to the tables.

In [3]:
# # First we need to get the JSON key for authentication

import urllib.request

url = 'https://storage.googleapis.com/angostura-public/hult-hackathon-key.json'
urllib.request.urlretrieve(url, './hult-hackathon-key.json')

#Second, we add the key to our environment

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './hult-hackathon-key.json'

# 3. Import GCloud API

In [4]:
from google.cloud import bigquery

client = bigquery.Client()

# 4. Example queries

In [5]:
# # We can dump the BigQuery results directly into a DataFrame
QUERY = ('select * from `angostura_dev`.eh_health_survey_response')

df = client.query(QUERY).to_dataframe()

In [6]:
print("Here are some quick details about the data:")
print(f">>  There are {len(df)} rows")
print(f">>  The shape of the df is {df.shape}")

Here are some quick details about the data:
>>  There are 1012 rows
>>  The shape of the df is (1012, 121)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Columns: 121 entries, timestamp to power_outage_equipment_failure
dtypes: float64(10), int64(16), object(95)
memory usage: 956.8+ KB


In [8]:
# Quick options to display more data in the output cells
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df.head(100)

In [ ]:
df.to_excel('data_2.xlsx')

In [ ]:
df.axes

In [ ]:
df.isnull().sum()

In [17]:
#new_data = pd.read_excel('data.xlsx')
new_data = df.copy()

# Data Cleaning

In [18]:
new_data.isnull().sum()

timestamp                                            0
report_week                                          0
operability_icu                                      0
power_outage_equipment_failure_specify               0
power_generator_available                            0
power_outage_mortatility                             0
power_outage_deaths_count                          483
mortality_hospital_failure_cardiovascular_count    299
mortality_hospital_failure_cardiovascular_cause      0
mortality_hospital_failure_trauma_count            336
mortality_hospital_failure_trauma_cause              0
strike_medical_staff_affected                        0
strike_nurses_affected                               0
operability_icu_p                                    0
strike_other_staff_affected                          0
strike_patients_affected                             0
strike_other_affected                                0
staff_violence_affected_reasons                      0
epidemiolo

In [19]:
new_data['rrt_num_hemodialysis_acute'][:30]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
Name: rrt_num_hemodialysis_acute, dtype: float64

In [58]:
import numpy as np

replace_dict = {'Nunca ha existido' : np.nan,
                'No operativa' : 0,
                "Sí" : 1,
                "No" : 0,
                'Nunca ha existido': np.nan,
                'No hay': 0,
                "Menos de 3 días": 1,
                "Entre 3 y 5 días": 2,
                "Todos los días":3,
                "Ningún día" : 0,
                "Entre 1 y 2 días" : 1,
                "Entre 3 y 5 días" : 2,
                "Todos los días" : 3,
                "Protestas sin suspensión de actividades" : 0,
                "Protestas con suspensión temporal de actividades" : 1,
                "Huelga (suspendidas todas las actividades)" : 2,
                "con especilaista" : 1,
                "sin especialista" : 0,
                "No hubo" : 0,
                "No se siguen recomendaciones del especialista en cuanto al menú que se sirve" : 0,
                'Se siguen recomendaciones del especialista en cuanto al menú que se sirve' : 1,
                'Si' : 1,
                'Hay pero no funciona' : 0,
                'Funciona menos de 3 días' : 1,
                'Funciona entre 3 y 5 días' : 2,
                "Funciona todos los día's" : 3,
                 "Funciona todos los días" : 3,
                'Nunca ha habido' : np.nan,
                'No existe' : np.nan,
                "No hubo agua ningún dia" : 0,
                "< 3 días, con soporte alterno" : 2,
                "< 3 días, sin soporte alterno (cisternas)" : 1,
                "3 a 5 días , sin soporte alterno" : 3,
                "3 a 5 días, con soporte alterno" : 4,
                "Hubo agua todos los días" : 5,
                 }

new_data = new_data.replace(replace_dict)

# Replacing Blank Spaces in nCoV columns
ncov_cols = ["nCoV_respiratory_isolation_protocol_avail",
"nCoV_isolation_area_avail"]

new_data[ncov_cols] = new_data[ncov_cols].replace({'':np.nan})

In [61]:
#new_data.to_excel('tableau_data3.0.xlsx')

#new_data.to_csv('tableau_data4.0.xlsx')

In [ ]:
operability_icu
power_outage_equipment_failure_specify
power_generator_available
power_outage_mortatility
operability_icu_p
rrt_avail_high_flow_catheters
rrt_avail_blood_tests_hiv_hvb_hvc_vdr
rrt_avail_immediate_access_urea_reduction_bun
nCoV_face_mask_avail
nCoV_respiratory_isolation_protocol_avail
operability_er
nCoV_isolation_area_avail
operability_sx
operability_lab
operability_uls
operability_ct_mri
operability_xr
er_avail_adrenalin
er_avail_atropine
er_avail_dopamine
er_avail_cephalosporins_betalactams
er_avail_aminoglycosides_quinolone
er_avail_vancomycin_clindamycin
er_avail_lidocaine
er_avail_minor_opioids
er_avail_major_opioids
er_avail_iv_fluids
er_avail_diazepam_dph
er_avail_heparin
er_avail_steroids
er_avail_insulin
er_avail_asthma
er_avail_blood_pressure
er_avail_defibrillator
er_avail_ott_intubation
er_avail_catheter
er_avail_oxygen_suction
sx_avail_minor_opioids
sx_avail_major_opioids
sx_avail_anesthetic_gases
sx_avail_anesthetics_iv
sx_avail_relaxants
sx_avail_ott_intubation
sx_avail_patient_lingerie_kit
sx_avail_disposables_mask_gloves_gown
sx_avail_oxygen_suction
rrt_avail
rrt_operability
rrt_hemodialysis_avail_filter
rrt_hemodialysis_avail_lines
rrt_hemodialysis_avail_kit_hemodialysis
rrt_hemodialysis_avail_iron
rrt_hemodialysis_avail_b_complex
rrt_hemodialysis_avail_calcium
rrt_hemodialysis_avail_zemblar
rrt_reverse_osmosis_unit_operability
rrt_avail_blood_tests_hiv_hvb_hvc_vdr
rrt_avail_immediate_access_urea_reduction_bun
rrt_avail_high_flow_catheters
rrt_staff_nephrology
rrt_staff_md
rrt_staff_resident
rrt_staff_nurse
rrt_staff_nurse_nephrologist
nutr_avail
nutr_operability
nutr_daily_freq_meal
op_pavilions_count
power_outage
power_outage_days_count
power_outage_equipment_failure

## Scarcity Scores
### Dialysis Supplies Scarcity

In [82]:
RRT_cols = []
for col in new_data:
    if col.find("rrt_avail_")==0 or col.find("rrt_hemo")==0:
        RRT_cols.append(col)
        new_data[col] = pd.to_numeric(new_data[col])
RRT_cols

new_data['RRT_Scarcity'] = 100-(((new_data[RRT_cols].sum(axis=1))/(len(RRT_cols)*3))*100)

In [83]:
new_data['RRT_Scarcity']

0       100.000000
1       100.000000
2       100.000000
3       100.000000
4       100.000000
           ...    
1007     80.000000
1008     70.000000
1009     90.000000
1010     10.000000
1011     83.333333
Name: RRT_Scarcity, Length: 1012, dtype: float64

### Emergency Room Supplies Scarcity

In [94]:
ER_cols = []
for col in new_data:
    if col.find("er_avail")==0: # or col.find("rrt_hemo")==0:
        ER_cols.append(col)
        new_data[col] = pd.to_numeric(new_data[col])

new_data['ER_Scarcity'] = 100-(((new_data[ER_cols].sum(axis=1))/(len(ER_cols)*3))*100)
new_data['ER_Scarcity']

### Surgery Supplies Scarcity

In [98]:
SX_cols = []
for col in new_data:
    if col.find("sx_avail")==0: # or col.find("rrt_hemo")==0:
        SX_cols.append(col)
        new_data[col] = pd.to_numeric(new_data[col])

new_data['SX_Scarcity'] = 100-(((new_data[SX_cols].sum(axis=1))/(len(SX_cols)*3))*100)
new_data['SX_Scarcity']

0       33.333333
1       44.444444
2       37.037037
3       40.740741
4       33.333333
          ...    
1007    44.444444
1008    33.333333
1009    44.444444
1010     0.000000
1011    85.185185
Name: SX_Scarcity, Length: 1012, dtype: float64

In [106]:
#new_data['op_beds_count'].max().groupby('hospital_code')

In [ ]:
# for col in new_data:
#     if new_data[col].isnull().sum() > 50:
#         new_data = new_data.drop(col,axis=1)
        

In [ ]:
#new_data.to_excel('data_without_missing.xlsx')

In [ ]:

# for col in list(new_data.columns.values):
#     if len(new_data[col].value_counts())>= 1:
#         new_data = pd.concat([new_data, pd.get_dummies(new_data[col], prefix=col)], axis=0)
#         #new_data[col].fillna(0,inplace=True)
    
        
        
#         #new_data = new_data.drop(col,axis=1)

In [ ]:
# for col in new_data:
#     new_data[f'one_hot_{col}'] = pd.get_dummies()

# 5. Data Format and Conventions

All the data comes from forms that doctors in all public hospitals in Venezuela fill on a daily basis.
Here we explain the format and conventions used

## Columns Details
|Variable|Name|Range|Type|
|:-------|:---|:----|:---|
|timestamp|Timestamp|MM/DD/YYYY HH:MM:SS|Timestamp|
|report_week|Semana reportada|[See Weeks below](#weeks)|String|
|hospital_code|1.- Código del Hospital|[See Hospitals List below](#hospitals)|String|
|federal_entity|2.- Entidad Federal|[See Hospitals List below](#hospitals)|String|
|hospital_type|3.- Tipo de Hospital|II, III, IV|String|
|administrative_entity|4.- Ente Administrativo (selecciones todas las opciones que corresponda)|MMPS, IVSS, SM, Estatal, Barrio Adentro, Autonomo |String|
|arch_beds_count|5.- Número de camas arquitectónicas|Numero entero|Numeric|
|op_beds_count|6.- Número de camas operativas|Numero entero|Numeric|
|op_beds_er_count|7.- Número de camas operativas en emergencia|Numero entero|Numeric|
|op_pavilions_count|8.- Número de pabellones  operativos|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_icu|9.- Operatividad de unidades específicas [UTI]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_icu_p|9.- Operatividad de unidades específicas [UTI(P)]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_er|9.- Operatividad de unidades específicas [Emergencia]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_sx|9.- Operatividad de unidades específicas [Pabellón]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_lab|9.- Operatividad de unidades específicas [Laboratorio]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_uls|9.- Operatividad de unidades específicas [Ecografía]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_ct_mri|9.- Operatividad de unidades específicas [TAC/RMN]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|operability_xr|9.- Operatividad de unidades específicas [Rayos x]|Nunca ha existido, No operativa, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_adrenalin|10.- Señale insumos disponibles en emergencia. Lista de insumos [Adrenalina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_atropine|10.- Señale insumos disponibles en emergencia. Lista de insumos [Atropina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_dopamine|10.- Señale insumos disponibles en emergencia. Lista de insumos [Dopamina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_cephalosporins_betalactams|10.- Señale insumos disponibles en emergencia. Lista de insumos [Cefalosporinas /betalactámicos]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_aminoglycosides_quinolone|10.- Señale insumos disponibles en emergencia. Lista de insumos [Aminoglicósidos / quinolonas]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_vancomycin_clindamycin|10.- Señale insumos disponibles en emergencia. Lista de insumos [Vancomicina / Clindamicina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_lidocaine|10.- Señale insumos disponibles en emergencia. Lista de insumos [Anestesia local (lidocaína)]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_minor_opioids|10.- Señale insumos disponibles en emergencia. Lista de insumos [Analgésico menor ( AINES , dipirona , etc)]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_major_opioids|10.- Señale insumos disponibles en emergencia. Lista de insumos [Analgésico mayor ( morfina , demerol)]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_iv_fluids|10.- Señale insumos disponibles en emergencia. Lista de insumos [Fluidoterápicos ( 0,9% , Gluc 5%, ) / equipos infusión]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_diazepam_dph|10.- Señale insumos disponibles en emergencia. Lista de insumos [Diazepam/ DPH]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_heparin|10.- Señale insumos disponibles en emergencia. Lista de insumos [Heparina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_steroids|10.- Señale insumos disponibles en emergencia. Lista de insumos [Esteroides]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_insulin|10.- Señale insumos disponibles en emergencia. Lista de insumos [Insulina]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_asthma|10.- Señale insumos disponibles en emergencia. Lista de insumos [Med inhalada para asma]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_blood_pressure|10.- Señale insumos disponibles en emergencia. Lista de insumos [Antihipertensivo]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_defibrillator|10.- Señale insumos disponibles en emergencia. Lista de insumos [Desfibrilador]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_ott_intubation|10.- Señale insumos disponibles en emergencia. Lista de insumos [TOT / intubación]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_catheter|10.- Señale insumos disponibles en emergencia. Lista de insumos [Yelco / vías centrales]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|er_avail_oxygen_suction|10.- Señale insumos disponibles en emergencia. Lista de insumos [Oxígeno / succión]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_minor_opioids|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Analgésicos menores]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_major_opioids|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Analgésicos mayores]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_anesthetic_gases|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Gases anestésicos]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_anesthetics_iv|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Anestésicos EV]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_relaxants|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Relajantes]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_ott_intubation|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [TOT / equipo intubación ]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_patient_lingerie_kit|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Kit lencería paciente]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_disposables_mask_gloves_gown|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Desechables personal de salud( tapaboca, guantes , bata )]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|sx_avail_oxygen_suction|11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos  [Oxígeno / succión]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_avail|12.- ¿Existe servicio de diálisis en su hospital?|Si, No|String|
|rrt_operability|13.-¿Está operativo el servicio de diálisis?|Si, No|String|
|rrt_date_stopped_operability|14.- ¿Desde cuándo no está operativo el servicio de diálisis?|Mas de 1 año, Entre 6 meses y 1 Año, Entre 3 y 6 meses, Entre uno y  3 meses, Entre una semana y 1 mes, Menos de una semana|String|
|rrt_reason_stopped_operability|15.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de diálisis? (seleccione las correctas)|[See RRT Stopped Reason below](#rrt_reason_stopped_operability)|String|
|rrt_avg_daily_patients|16.- ¿Cuál es la cantidad de pacientes promedio diario en diálisis en el servicio en la última semana?|Número  entero|Numeric|
|rrt_peritoneal_count|17.- ¿Cuántos de estos están en diálisis peritoneal?|Número  entero|Numeric|
|rrt_peritoneal_reason_not_performed|18.- Si la respuesta a la pregunta anterior es "0" ¿Cuál es la/las razón(es) por la que no se realiza  diálisis peritoneal? (seleccione las correctas)|[See RRT No Dialysis below](#rrt_peritoneal_reason_not_performed)|String|
|rrt_num_hemodialysis|19.- ¿Cuántos de estos están en hemodiálisis?|Número  entero|Numeric|
|rrt_num_hemodialysis_acute|20.-¿Cuántos de estos pacientes son agudos?|Número  entero|Numeric|
|rrt_num_hemodialysis_chronic|21.-¿Cuántos de estos pacientes son crónicos?|Número  entero|Numeric|
|rrt_num_hemodialysis_equipments|22.-¿Cuántos equipos de hemodiálisis existen en el servicio?|Número  entero|Numeric|
|rrt_num_hemodialysis_equipments_operability|23.-¿Cuántos de estos equipos de hemodiálisis están operativos?|Número  entero|Numeric|
|rrt_hemodialysis_avail_filter|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Filtros]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_lines|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Lineas de conexión]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_kit_hemodialysis|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Kit de hemodiálisis ]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_iron|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Hierro]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_b_complex|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Complejo B]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_calcium|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Calcio]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_hemodialysis_avail_zemblar|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Zemblar®]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_reverse_osmosis_unit_operability|25.-Con respecto a la planta de ósmosis inversa, ¿Cuántos días estuvo operativa la última semana?|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_staff_nephrology|26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Especialista Nefrólogo]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_staff_md|26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Médico general]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_staff_resident|26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Residente]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_staff_nurse|26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Enfermera graduada]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_staff_nurse_nephrologist|26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Enfermera especialista en nefrología]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|nutr_avail|27.- ¿Existe servicio de nutrición en su hospital?|Si, No|String|
|nutr_operability|28.-¿Está operativo el servicio de nutrición?|Si, No|String|
|nutr_date_stopped_operability|29.- ¿Desde cuándo no está operativo el servicio de nutrición?|Mas de 1 año, Entre 6 meses y 1 Año, Entre 3 y 6 meses, Entre uno y  3 meses, Entre una semana y 1 mes, Menos de una semana|String|
|nutr_reason_stopped_operability|30.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de nutrición? (seleccione las correctas)|[See NUTR Reason Stopped below](#nutr_reason_stopped_operability)|String|
|nutr_num|31.- ¿Cuántos días funcionó el servicio de nutrición la última semana?|Ningún día, Entre 1 y 2 días, Entre 3 y 5 días, Todos los días|String|
|nutr_daily_freq_meal|32.- Frecuencia diaria de comidas del servicio de nutrición|3 comidas, menos de 3 comidas|String|
|nutr_quality|33.- Calidad del servicio de nutrición|con especilaista, sin especialista|String|
|nutr_freq_milk_formulas|34. - Frecuencia en el suministro de fórmulas lácteas (solo si existe pediatría)|[See NUTR Milk Formulas Frequency below](#nutr_freq_milk_formulas)|String|
|pneumonia_antibiotic_therapy_latency|35.- En su última guardia, en un paciente que requiere hospitalización por neumonía, estime el tiempo que pasó entre la llegada a emergencia y colocación de 1ra dosis de antibiótico|HH:MM|Time|
|mi_thrombolytic_treatment_latency|36.- En su última guardia, en un paciente con infarto al miocardio agudo, estime el intervalo de tiempo entre llegada y uso de trombolítico / heparina|HH:MM|Time|
|er_staff_residents_and_rural_day_on_call|37.- Número de médicos residentes o rurales  en el turno diurno|Número  entero|Numeric|
|er_staff_specialist_day_on_call|38.- Número de médicos especialistas en el turno diurno|Número  entero|Numeric|
|er_staff_mic_day_on_call|39.- Número de MIC en el turno diurno|Número  entero|Numeric|
|er_staff_nurse_day_on_call|40.- Número de enfermeras profesionales en el turno diurno|Número  entero|Numeric|
|er_staff_non_professional_nurse_day_on_call|41.- Número de enfermeras no profesionales en el turno diurno|Número  entero|Numeric|
|er_staff_residents_and_rural_night_on_call|42.- Número de médicos residentes o rurales en el turno nocturno|Número  entero|Numeric|
|er_staff_specialist_night_on_call|43.- Número de médicos especialistas en el turno nocturno|Número  entero|Numeric|
|er_staff_mic_night_on_call|44.- Número de MIC en el turno nocturno|Número  entero|Numeric|
|er_staff_nurse_night_on_call|45.- Número de enfermeras profesionales en el turno nocturno|Número  entero|Numeric|
|er_staff_non_professional_nurse_night_on_call|46.- Número de enfermeras no profesionales en el turno nocturno|Número  entero|Numeric|
|wash_failure_icu|47.- Fallas en servicio de agua en: [UTI]|[See Wash below](#wash)|String|
|wash_failure_er|47.- Fallas en servicio de agua en: [Emergencia]|[See Wash below](#wash)|String|
|wahs_failure_sx|47.- Fallas en servicio de agua en: [Pabellón]|[See Wash below](#wash)|String|
|power_outage|48.- Fallas en servicio de luz eléctrica.|Si, No|String|
|power_outage_avg_failures_per_day|49.- Número promedio de cortes por día:|Número  entero|Numeric|
|power_outage_days_count|50.- Número de días en los que hubo cortes de energía eléctrica|Menos de 3 días, Entre 3 y 5 días, Todos los días|String|
|power_outage_avg_duration|51.- Duración promedio de fallos por semana del servicio luz eléctrica|HH:MM:SS|Time|
|power_outage_equipment_failure|52.- ¿Hubo fallas en equipos posterior a cortes de energía la semana pasada?|SI, No|String|
|power_outage_equipment_failure_specify|53.- ¿Cuáles equipos fallaron posterior a fallas de energía la semana pasada?|Abierta|String|
|power_generator_available|54.- Planta eléctrica|SI, No|String|
|power_outage_mortatility|55.- ¿Hubo fallecidos por causas atribuibles a la falta de energía eléctrica?|SI, No|String|
|power_outage_deaths_count|56.-¿Cuántos fallecidos hubo por causas atribuibles a la falta de energía eléctrica?|Número  entero|Numeric|
|mortality_hospital_failure_cardiovascular_count|57.- Número de muertes cardiovasculares interpretada como falla institucional (emergencia )|Número  entero|Numeric|
|mortality_hospital_failure_cardiovascular_cause|58.- Causa de muerte cardiovascular interpretada como falla institucional (emergencia )|Equipos, medicamentos, recurso humano, luz eléctrica, otros estructurales, otras causas|String|
|mortality_hospital_failure_trauma_count|59.- Número de muertes por trauma interpretada como falla institucional (emergencia )|Número  entero|Numeric|
|mortality_hospital_failure_trauma_cause|60.- Causa de muerte por trauma agudo interpretada como falla institucional (emergencia )|Equipos, medicamentos, recurso humano, luz eléctrica, otros estructurales, otras causas|String|
|strike_medical_staff_affected|61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Personal médico]|Protestas  sin suspensión de actividades, Protestas con suspensión temporal  de actividades, Huelga (suspendidas todas  las actividades)|String|
|strike_nurses_affected|61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Enfermeras]|Protestas  sin suspensión de actividades, Protestas con suspensión temporal  de actividades, Huelga (suspendidas todas  las actividades)|String|
|strike_other_staff_affected|61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Otro personal del Hospital]|Protestas  sin suspensión de actividades, Protestas con suspensión temporal  de actividades, Huelga (suspendidas todas  las actividades)|String|
|strike_patients_affected|61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Familiares y/o pacientes]|Protestas  sin suspensión de actividades, Protestas con suspensión temporal  de actividades, Huelga (suspendidas todas  las actividades)|String|
|strike_other_affected|61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Otros]|Protestas  sin suspensión de actividades, Protestas con suspensión temporal  de actividades, Huelga (suspendidas todas  las actividades)|String|
|staff_violence_affected_reasons|62.- En caso que se hayan producido situaciones de violencia contra personal hospitalario, señale las opciones correspondientes (seleccione todas las opciones que corresponda)|[See STAFF Violence below](#staff_violence_affected_reasons)|String|
|epidemiological_emergency_suspected_diseases|63.- Señale alguna situación (fallecidos u hospitalizados) con sospecha clínica de cualquiera de estas enfermedades (seleccione todas las opciones que corresponda)|[See EPIDEMIOLOGICAL below](#epidemiological_emergency_suspected_disease)|String|
|rrt_avail_high_flow_catheters|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Catéteres de alto flujo]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_avail_blood_tests_hiv_hvb_hvc_vdr|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Exámenes de serología (HIV, HVB, HVC y VDR)L)]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|rrt_avail_immediate_access_urea_reduction_bun|24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Acceso inmediato a % de reducción de urea o BUN]|Nunca ha existido, No hay, Menos de 3 de días, Entre 3 y 5 días, Todos los días|String|
|nCoV_face_mask_avail|64.-¿Se suministra tapabocas al personal de salud en este hospital?|Si, No|String|
|nCoV_respiratory_isolation_protocol_avail|65.-¿Existe protocolo de aislamiento respiratorio específico?|Si, No|String|
|nCoV_isolation_area_avail|66.-¿Existe un área destinada para el aislamiento de casos sospechosos?|Si, No|String|

### Acronyms Dictionary
| Acronym | Definition |
|:--------|:-----------|
|arch|
|epidemiological|
|er| Emergency Room 
|mi| Myocardial infarction
|mortality|
|nCoV|
|nutr| Nutrition
|op|
|operability|
|pneumonia|
|power|
|rrt| Renal replacement therapy 
|staff|
|strike|
|sx| Surgery
|wash| water, sanitation and hygiene



### Hospitals 
|Hospital |hospital_code    | federal_entity |
|:---------|:------- |:----- |
| Hospital Dr. José Gregorio Hernández. Amazonas | AMA000 | Amazonas |
| Hospital Universitario Dr. Luis Razzetti. Anzoátegui | ANZ000 | Anzoátegui |
| Hospital Felipe Guevara Rojas. Anzoátegui | ANZ001 | Anzoátegui |
| Hospital de Guaraguao. Anzoátegui | ANZ002 | Anzoátegui |
| Hospital Dr. Pablo Acosta Ortiz. Apure | APU000 | Apure |
| Hospital José María Benítez. Aragua | ARA000 | Aragua | 
| Hospital Coronel Elbano Paredes Vivas. Aragua | ARA001 | Aragua |
| Hospital Central de Maracay. Aragua | ARA002 | Aragua | 
| Hospital Dr. Luis Razetti. Barinas | BAR000 | Barinas |
| Hospital Ruiz y Páez. Bolívar | BOL000 | Bolívar |
| Hospital Uyapar. Bolívar | BOL001 | Bolívar | 
| Hospital Dr. Ángel Larrralde. Carabobo | CAR000 | Carabobo |
| Ciudad Hospitalaria Enrique Tejera. Carabobo | CAR001 | Carabobo |
| Hospital General de San Carlos. Cojedes | COJ000 | Cojedes |
| Hospital Dr. Luis Razetti. Delta Amacuro | DEL000 | Delta Amacuro |
| Hospital Militar. Dtto. Capital | DCA000 | Distrito Capital |
| Hospital Vargas. Dtto. Capital | DCA001 | Distrito Capital |
| Hospital JM de los Ríos. Dtto. Capital | DCA002 | Distrito Capital |
| Hospital Universitario de Caracas. Dtto. Capital | DCA003 | Distrito Capital |
| Maternidad Concepción Palacios. Dtto. Capital | DCA004 | Distrito Capital |
| Hospital Dr. Miguel Pérez Carreño. Dtto. Capital | DCA005 | Distrito Capital |
| Hospital Magallanes de Catia. Dtto. Capital | DCA006 | Distrito Capital |
| Hospital Dr. Alfredo Van Grieken. Falcón | FAL000 | Falcón |
| Hospital Dr. Israel Ranuarez Balza. Guárico | GUA000 | Guárico | 
| Hospital Universitario Dr. Antonio María Pineda. Lara | LAR000 | Lara |
| Hospital Universitario de los Andes. Mérida | MER000 | Mérida |
| Hospital Domingo Luciani. Miranda | MIR000 | Miranda |
| Hospital General Dr. Victorino Santaella | MIR001 | Miranda |
| Hospital Universitario Dr. Manuel Núñez Tovar. Monagas | MON000 | Monagas |
| Hospital Dr. Luis Ortega. Nueva Esparta | NES000 | Nueva Esparta |
| Hospital Dr. Miguel Oraa. Portuguesa | POR000 | Portuguesa |
| Hospital Antonio Patricio de Alcalá. Sucre | SUC000 | Sucre |
| Hospital Patrocinio Peñuela. Táchira | TAC000 | Táchira | 
| Hospital Central de San Cristóbal. Táchira | TAC001 | Táchira |
| Hospital Universitario Dr. Pedro Emilio Carrillo. Trujillo | TRU000 | Trujillo |
| Hospital Dr. José María Vargas. Vargas | VAR000 | Vargas |
| Hospital Plácido Rodriguez Rivero , Yaracuy  | YAR000 | Yaracuy |
| Hospital Universitario de Maracaibo. Zulia | ZUL000 | Zulia |
| Hospital General del Sur | ZUL001 | Zulia

### RRT Stopped Reason
|rrt_reason_stopped_operability|
|:---|
|Falta de equipo nefrológico(máquinas de diálisis)|
|Falta de insumos médicos|
|Falta de profesionales especializados (Nefrólogo, Cirujano, etc)|
|Falta de material quirúrgico|
			
### RRT No Dyalisis
|rrt_peritoneal_reason_not_performed|
|:----------------------------------|
|Falta de insumos médicos|
|Falta de profesional entrenado (Cirujano)|
|Falta de material quirúrgico|

### NUTR Reason Stopped
|nutr_reason_stopped_operability|
|:------------------------------|
|Falta de equipos|
|Falta de profesionales (Nutricionistas, Nutrólogos etc)|
|Falta de insumos|
|Falta de insumos, Falta de equipos|
|Falta de personal|
	
	
### NUTR Milk Formulas Frequency
|nutr_freq_milk_formulas|
|:----------------------|
|Hubo fórmulas lácteas entre 3 y 5 días|
|Hubo formulas lácteas menos de 3 días|	
|Hubo fórmulas lácteas todos los días|	
|No hubo fórmulas lácteas ningún día|	
|Nunca ha habido fórmulas lácteas|


### WASH 
|wash_failure_icu, wash_failure_er, wahs_failure_sx|
|:-------------------------------------------------|
|< 3 días, con soporte alterno|
|< 3 días, sin soporte alterno (cisternas)|
|3 a 5 días , sin soporte  alterno|
|3 a 5 días, con soporte alterno|	
|Hubo agua todos los días|
|No hubo agua ningún dia|		


### STAFF Violence
|staff_violence_affected_reasons|
|:------------------------------|
|Robos , hurtos o disparos dentro del centro asistencial|
|Violencia contra personal de hospital por familiares|		
|Violencia contra personal por  grupos paramilitares|		
|Violencia contra personal por fuerzas de seguridad|
|Homicidios a personl de salud|

### EPIDEMIOLOGICAL 
|epidemiological_emergency_suspected_diseases|
|:-------------------------------------------|
|Difteria|
|Sarampión|
|Síndrome Congénito Zika|
|Síndrome Gillan Barré|
|TBC en personal de salud|
|Fiebre hemorrágica severa (UTI)|
|Influenza (UTI)|